In [1]:
n = 1 * int(1e4)
pbs = 32
q = 0.5

alpha = 1e-9 # failure prob.

from scipy.stats import binom

k = 1
binom_dist = binom(n, q)
while True:
    right_prob = binom_dist.sf(k * pbs)
    if right_prob < alpha:
        break
    k += 1

In [2]:
binom_dist.sf(k * pbs)

2.0263387747110406e-10

In [3]:
binom_dist.sf((k-1) * pbs)

1.0033293710926755e-08

In [4]:
k * pbs

5312

In [28]:
k

166

In [12]:
import jax
import jax.numpy as jnp
import numpy as np
from jax.lax import fori_loop
mlbs = k * pbs
Xs = jax.random.normal(jax.random.PRNGKey(123), (n,))

In [13]:
Xs

Array([ 0.7200799 ,  0.29004744,  0.3784297 , ...,  0.06939417,
       -0.8384669 , -0.55408597], dtype=float32)

In [14]:
mlbs

5312

In [16]:
choice_rng, binom_rng = jax.random.split(jax.random.PRNGKey(0), 2)
logical_batch = jax.random.choice(choice_rng, Xs, shape=(mlbs,), replace=False)
physical_batches = jnp.array(jnp.split(logical_batch, k))

actual_logical_bs = jax.random.bernoulli(binom_rng, q, shape=(n,)).sum()
masks = jnp.array(jnp.split((jnp.arange(mlbs) < actual_logical_bs), k))

In [18]:
type(logical_batch)

jaxlib.xla_extension.ArrayImpl

In [11]:
actual_logical_bs

Array(24962, dtype=int32)

In [17]:
len(masks)

803

In [18]:
len(masks[0])

32

In [24]:
(32*803)-actual_logical_bs

Array(734, dtype=int32)

In [32]:
for i,mb in enumerate(masks):
    if mb.all() == False:
        print(i,len(mb))

780 32
781 32
782 32
783 32
784 32
785 32
786 32
787 32
788 32
789 32
790 32
791 32
792 32
793 32
794 32
795 32
796 32
797 32
798 32
799 32
800 32
801 32
802 32


In [7]:
def single_iter(t_iter, args):
    choice_rng, binom_rng = jax.random.split(jax.random.PRNGKey(t_iter), 2)
    logical_batch = jax.random.choice(choice_rng, Xs, shape=(mlbs,), replace=False)
    physical_batches = jnp.array(jnp.split(logical_batch, k))
    
    actual_logical_bs = jax.random.bernoulli(binom_rng, q, shape=(n,)).sum()
    masks = jnp.array(jnp.split((jnp.arange(mlbs) < actual_logical_bs), k))

    def foo(t, args):
        cumulative_sum_so_far = args
        mask = masks[t]
        pb_sum = (physical_batches[t] * mask).sum()
        return cumulative_sum_so_far + pb_sum

    final_sum = fori_loop(0, k, foo, 0.)
    
    # add noise to grads
    # update parameters
    # instead of final sum, you would pass the updated params, or whatever to the next iteration
    return final_sum
final_result = fori_loop(0, 10, single_iter, 0.)

In [8]:
final_result

Array(85.314644, dtype=float32)

In [73]:
import flax.linen as nn
import optax
from flax.core.frozen_dict import freeze,unfreeze,FrozenDict
import functools

In [59]:
main_key, params_key= jax.random.split(key=jax.random.PRNGKey(1),num=2)

In [60]:
class CNN(nn.Module):
    """A simple CNN model."""

    @nn.compact
    def __call__(self, x):
        x = nn.Conv(features=64, kernel_size=(7, 7),strides=2)(x)
        x = nn.relu(x)
        x = nn.avg_pool(x, window_shape=(2, 2), strides=(2, 2))
        #x = nn.Conv(features=64, kernel_size=(3, 3))(x)
        #x = nn.relu(x)
        #x = nn.avg_pool(x, window_shape=(2, 2), strides=(2, 2))
        x = x.reshape((x.shape[0], -1))  # flatten
        x = nn.Dense(features=256)(x)
        x = nn.relu(x)
        x = nn.Dense(features=100)(x)
        return x

model = CNN()
input_shape = (1,3,32,32)
#But then, we need to split it in order to get random numbers


#The init function needs an example of the correct dimensions, to infer the dimensions.
#They are not explicitly writen in the module, instead, the model infer them with the first example.
x = jax.random.normal(params_key, input_shape)

main_rng, init_rng, dropout_init_rng = jax.random.split(main_key, 3)
#Initialize the model
variables = model.init({'params':init_rng},x)
#variables = model.init({'params':main_key}, batch)
model.apply(variables, x)
model = model
params = variables['params']

In [72]:
params = unfreeze(params)

In [61]:
import torch
import torchvision
import torch.utils.data as data
import numpy as np

DATA_MEANS2 = (0.485, 0.456, 0.406)
DATA_STD2 =  (0.229, 0.224, 0.225)

def image_to_numpy_wo_t(img):
    img = np.array(img, dtype=np.float32)
    img = ((img / 255.) - DATA_MEANS2) / DATA_STD2
    img = np.transpose(img,[2,0,1])
    return img

transformation = torchvision.transforms.Compose([
        #torchvision.transforms.Resize(224),
        image_to_numpy_wo_t,
        #torchvision.transforms.ToTensor(),
        #torchvision.transforms.Normalize(DATA_MEANS,DATA_STD),
    ])

train_set = torchvision.datasets.CIFAR10(root='../data_cifar10/',train=True,download=True,transform=transformation)
train_loader = data.DataLoader(train_set,batch_size=k*pbs,shuffle=True)

Files already downloaded and verified


In [62]:
mlbs

5312

In [51]:
acc_sum = 0
for batch_idx,(x,y) in enumerate(train_loader): #logical

    print(batch_idx)
    #print(type(x),x[0].shape)
    x  = jnp.array(x)
    y = jnp.array(y)
    print(type(x),len(x),x.shape)
    print(type(y),len(y),y.shape)

    diff = len(y) % k

    if diff > 0:

        x = jnp.pad(x, ((0, k - diff), (0, 0), (0, 0), (0, 0)), mode='constant')
        y = jnp.pad(y, ((0, k - diff)), mode='constant')
        print('new shape',x.shape,y.shape)
    
    batch_size = len(x)

    choice_rng, binom_rng = jax.random.split(jax.random.PRNGKey(batch_idx), 2)

    physical_batches = jnp.array(jnp.split(x, k))
    physical_labels = jnp.array(jnp.split(y,k))
    actual_logical_bs = jax.random.bernoulli(binom_rng, q, shape=(n,)).sum()
    masks = jnp.array(jnp.split((jnp.arange(batch_size) < actual_logical_bs), k))

    b = physical_batches[802]*masks[802]
    print(b)
    break

    print(len(masks))
    print(actual_logical_bs)
    def foo(t, args):
        cumulative_sum_so_far = args
        mask = masks[t]
        pb_sum = (physical_labels[t] * mask).sum()
        return cumulative_sum_so_far + pb_sum

    final_sum = fori_loop(0, k, foo, 0.)
    print('final sum',batch_idx,final_sum)
    acc_sum += final_sum
print('final sum',acc_sum)

0
<class 'jaxlib.xla_extension.ArrayImpl'> 5312 (5312, 3, 32, 32)
<class 'jaxlib.xla_extension.ArrayImpl'> 5312 (5312,)
[[[[-0. -0. -0. ...  0. -0.  0.]
   [-0. -0. -0. ...  0.  0.  0.]
   [-0. -0. -0. ...  0.  0.  0.]
   ...
   [-0. -0. -0. ...  0.  0.  0.]
   [-0. -0. -0. ...  0.  0.  0.]
   [-0. -0. -0. ...  0.  0.  0.]]

  [[ 0.  0.  0. ...  0.  0.  0.]
   [ 0.  0.  0. ...  0.  0.  0.]
   [ 0.  0.  0. ...  0.  0.  0.]
   ...
   [ 0.  0.  0. ...  0.  0.  0.]
   [ 0.  0.  0. ...  0.  0.  0.]
   [ 0.  0.  0. ...  0.  0.  0.]]

  [[ 0.  0.  0. ...  0.  0.  0.]
   [ 0.  0.  0. ...  0.  0.  0.]
   [ 0.  0.  0. ...  0.  0.  0.]
   ...
   [ 0.  0.  0. ...  0.  0.  0.]
   [ 0.  0.  0. ...  0.  0.  0.]
   [ 0.  0.  0. ...  0.  0.  0.]]]


 [[[-0. -0. -0. ... -0. -0. -0.]
   [-0. -0. -0. ... -0. -0. -0.]
   [-0. -0. -0. ... -0. -0. -0.]
   ...
   [-0. -0. -0. ... -0. -0. -0.]
   [-0. -0. -0. ...  0.  0.  0.]
   [-0. -0. -0. ...  0.  0.  0.]]

  [[-0. -0. -0. ... -0. -0. -0.]
   [-0. -0. -0. .

In [31]:
a = jnp.ones((5,))


In [37]:
jnp.pad(a, ((0, 2),), mode='constant')

Array([1., 1., 1., 1., 1., 0., 0.], dtype=float32)

In [63]:
def loss(params,batch):
    inputs,targets = batch
    logits = model.apply({'params':params},inputs)
    predicted_class = jnp.argmax(logits,axis=-1)

    cross_loss = optax.softmax_cross_entropy_with_integer_labels(logits, targets).mean()

    vals = predicted_class == targets
    acc = jnp.mean(vals)
    cor = jnp.sum(vals)

    return cross_loss,(acc,cor)

In [64]:
def non_private_update(params,batch):
    (loss_val,(acc,cor)), grads = jax.value_and_grad(loss,has_aux=True)(params,batch)
    return grads,loss_val,acc,cor

In [65]:
lr = 0.00031

In [66]:
optimizer = optax.adam(learning_rate=lr)
opt_state = optimizer.init(params)

In [67]:
def grad_acc_update(grads,opt_state,params):
    updates,opt_state = optimizer.update(grads,opt_state,params)
    params = optax.apply_updates(params,updates)
    return params,opt_state

In [75]:
acc_sum = 0
_acc_update = lambda grad, acc : grad + acc
for batch_idx,(x,y) in enumerate(train_loader): #logical

    print(batch_idx)
    #print(type(x),x[0].shape)
    x  = jnp.array(x)
    y = jnp.array(y)
    print(type(x),len(x),x.shape)
    print(type(y),len(y),y.shape)

    diff = len(y) % k

    if diff > 0:

        x = jnp.pad(x, ((0, k - diff), (0, 0), (0, 0), (0, 0)), mode='constant')
        y = jnp.pad(y, ((0, k - diff)), mode='constant')
        print('new shape',x.shape,y.shape)
    
    batch_size = len(x)

    choice_rng, binom_rng = jax.random.split(jax.random.PRNGKey(batch_idx), 2)

    physical_batches = jnp.array(jnp.split(x, k))
    physical_labels = jnp.array(jnp.split(y,k))
    actual_logical_bs = jax.random.bernoulli(binom_rng, q, shape=(n,)).sum()
    masks = jnp.array(jnp.split((jnp.arange(batch_size) < actual_logical_bs), k))
    acc_grads = jax.tree_util.tree_map(jnp.zeros_like,params)
    def foo(t, args):
        acc_grad = args
        mask = masks[t]
        data_x = (physical_batches[t] * mask)
        data_y = (physical_labels[t] * mask)
        grads,loss,acc,cor = non_private_update(params,(data_x,data_y))
        return jax.tree_util.tree_map(
                            functools.partial(_acc_update),
                            grads, acc_grad)

    accumulated_gradients = fori_loop(0, k, foo, acc_grads)
    print('update?',accumulated_gradients)
    params,opt_state = grad_acc_update(accumulated_gradients,opt_state,params)


0
<class 'jaxlib.xla_extension.ArrayImpl'> 5312 (5312, 3, 32, 32)
<class 'jaxlib.xla_extension.ArrayImpl'> 5312 (5312,)


ValueError: Expected dict, got FrozenDict({
    Conv_0: {
        bias: Array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
               0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
               0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
               0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32),
        kernel: Array([[[[0., 0., 0., ..., 0., 0., 0.],
                 [0., 0., 0., ..., 0., 0., 0.],
                 [0., 0., 0., ..., 0., 0., 0.],
                 ...,
                 [0., 0., 0., ..., 0., 0., 0.],
                 [0., 0., 0., ..., 0., 0., 0.],
                 [0., 0., 0., ..., 0., 0., 0.]],
        
                [[0., 0., 0., ..., 0., 0., 0.],
                 [0., 0., 0., ..., 0., 0., 0.],
                 [0., 0., 0., ..., 0., 0., 0.],
                 ...,
                 [0., 0., 0., ..., 0., 0., 0.],
                 [0., 0., 0., ..., 0., 0., 0.],
             